In [1]:
import pandas as pd
import numpy as np

In [2]:
original_features = pd.read_csv('./output/original_mask_features.csv', index_col=0)
name_mapping = pd.read_csv("mapping/name_mapping.csv")

In [3]:
volumes = original_features.loc[:, ["original_shape_MeshVolume"]]

In [4]:
new_volumes = pd.DataFrame(index=name_mapping["BraTS_2020_subject_ID"], columns=["original_shape_MeshVolume", "Size"])

In [5]:
for case_id in volumes.index:
    if "FLAIR" in case_id:
        new_name = case_id[0:20]
        new_volumes.loc[new_name] = volumes.loc[case_id]

In [6]:
Q1 = new_volumes.original_shape_MeshVolume.quantile([0.25]).to_numpy()[0]
Q1

3573.8125

In [7]:
Q3 = new_volumes.original_shape_MeshVolume.quantile([0.75]).to_numpy()[0]
Q3

25169.666666666668

In [8]:
for case_id in new_volumes.index:
    volume = float(new_volumes.loc[[case_id],['original_shape_MeshVolume']].values[0])
    if volume <= Q1:
        new_volumes.loc[[case_id],['Size']] = "SMALL"
    if volume > Q1 and volume < Q3:
        new_volumes.loc[[case_id],['Size']] = "MEDIUM"
    if volume >= Q3:
        new_volumes.loc[[case_id],['Size']] = "LARGE"

In [9]:
name_mapping["Size"] = new_volumes["Size"].to_list()

In [10]:
strat_values = []
for id in name_mapping.index:
    grade = str(name_mapping.loc[[id],['Grade']].values[0][0])
    size = str(name_mapping.loc[[id],['Size']].values[0][0])
    strat_value = grade + "_" + size
    strat_values.append(strat_value)

In [201]:
name_mapping["Stratification_Label"] = strat_values
del name_mapping["Size"]

In [202]:
name_mapping.to_csv("mapping/name_mapping.csv", index=False)

In [11]:
name_mapping

,Grade,BraTS_2020_subject_ID,Stratification_Label,Size
0,HGG,BraTS20_Training_001,HGG_MEDIUM,MEDIUM
1,HGG,BraTS20_Training_002,HGG_MEDIUM,MEDIUM
2,HGG,BraTS20_Training_003,HGG_SMALL,SMALL
3,HGG,BraTS20_Training_004,HGG_MEDIUM,MEDIUM
4,HGG,BraTS20_Training_005,HGG_SMALL,SMALL
...,...,...,...,...
363,HGG,BraTS20_Training_365,HGG_LARGE,LARGE
364,HGG,BraTS20_Training_366,HGG_MEDIUM,MEDIUM
365,HGG,BraTS20_Training_367,HGG_SMALL,SMALL
366,HGG,BraTS20_Training_368,HGG_SMALL,SMALL


In [12]:
del name_mapping["Stratification_Label"]

In [13]:
name_mapping

,Grade,BraTS_2020_subject_ID,Size
0,HGG,BraTS20_Training_001,MEDIUM
1,HGG,BraTS20_Training_002,MEDIUM
2,HGG,BraTS20_Training_003,SMALL
3,HGG,BraTS20_Training_004,MEDIUM
4,HGG,BraTS20_Training_005,SMALL
...,...,...,...
363,HGG,BraTS20_Training_365,LARGE
364,HGG,BraTS20_Training_366,MEDIUM
365,HGG,BraTS20_Training_367,SMALL
366,HGG,BraTS20_Training_368,SMALL


In [18]:
is_SMALL =  name_mapping['Size']=='SMALL'
is_MEDIUM =  name_mapping['Size']=='MEDIUM'
is_LARGE =  name_mapping['Size']=='LARGE'
small_cases = name_mapping[is_SMALL]
medium_cases = name_mapping[is_MEDIUM]
large_cases = name_mapping[is_LARGE]

In [23]:
small_cases.to_csv("mapping/small_mapping.csv", index=False)
medium_cases.to_csv("mapping/medium_mapping.csv", index=False)
large_cases.to_csv("mapping/large_mapping.csv", index=False)